In [1]:
import os
from lxml import etree
import gensim
#read the corpus
src_file = os.path.join('src/DLUG_Ann_clean.vrt')
tree = etree.parse(src_file)
root = tree.getroot()

In [2]:
#check type of text block
def check_type(div):
    typeof = 'text'  #p div etc
    if div.tag == "head":
        if "class" in div.attrib.keys():
            if div.attrib["class"] == "annus":
                typeof='annus'
            elif div.attrib["class"] == "chapter":
                typeof='chapter'
            else:
                typeof="head"
        else:
                typeof="head"                
    return typeof

In [3]:
def process_vrt(raw):
    lines = []
    for line in [line for line in raw.split("\n") if line != ""]:
        tok,pos,lem = None, None, None
        cols = line.split('\t')
        if len(cols) == 3:
            tok, pos,lem = cols
        else:
            tok = cols[0]
        lines.append((tok,pos,lem))
    return lines

In [4]:
text_dict = {}
#text_dict = {book : {chapter : [{year, typeof, txt}]} }
liber = 0
for el in root: #liber
    year = 0
    chapter = None
    classes = ['head', 'annus', 'text']
    typeof = 'text'
    raw_text = ''
    
    #liber
    if "class" in el.attrib.keys():
        if el.attrib["class"] == 'liber':
            liber = int(el.attrib["n"])
            text_dict[liber] = dict()
            #print("liber",liber)
    for ell in el.getchildren():
        if "class" in el.attrib.keys():
            if ell.attrib["class"] == "year":
                year = ell.attrib["n"]
                #print("year ", year)
                for elll in ell.getchildren():
                    if elll.tag == "head":
                        typeof = check_type(elll)                
                        raw_text = elll.text
                        
                        if chapter is not None:
                            chapter += 1
                        else:
                            chapter = 0
                        
                        text_dict[liber][chapter] = list()
                        text_dict[liber][chapter].append([year, typeof, process_vrt(raw_text)])
                            
                    elif elll.tag == "div":
                        if elll.attrib["class"] == "chapter":
                            if chapter is not None:
                                chapter += 1
                            else:
                                chapter = 0
                            #print("chapter ", chapter)
                            text_dict[liber][chapter] = list()
                            for ellll in elll.getchildren():
                                typeof = check_type(ellll)
                                raw_text = ellll.text
                                #print("chapter ", chapter)                           
                                text_dict[liber][chapter].append([year, typeof, process_vrt(raw_text)])
                                
                                
                    
            #chapters under liber 0-1
            elif ell.attrib["class"] == "chapter":
                if chapter is not None:
                    chapter += 1
                else:
                    chapter = 0
                text_dict[liber][chapter] = list()
                for elll in ell.getchildren():
                    typeof = check_type(elll)
                    raw_text = elll.text
                    text_dict[liber][chapter].append([year, typeof, process_vrt(raw_text)])

In [5]:
text_dict = {} #{book : {chapter : [{year, typeof, txt}]} }
liber = 0
for el in root: #liber
    year = 0
    chapter = None
    classes = ['head', 'annus', 'text']
    typeof = 'text'
    raw_text = ''
    
    #liber
    if "class" in el.attrib.keys():
        if el.attrib["class"] == 'liber':
            liber = int(el.attrib["n"])
            text_dict[liber] = dict()
            #print("liber",liber)
    for ell in el.getchildren():
        if "class" in el.attrib.keys():
            if ell.attrib["class"] == "year":
                year = ell.attrib["n"]
                #print("year ", year)
                for elll in ell.getchildren():
                    if elll.tag == "head":
                        typeof = check_type(elll)                
                        raw_text = elll.text
                        if chapter is not None:
                            chapter += 1
                        else:
                            chapter = 0
                        text_dict[liber][chapter] = list()
                        text_dict[liber][chapter].append([year, typeof, process_vrt(raw_text)])
                            
                    elif elll.tag == "div":
                        if elll.attrib["class"] == "chapter":
                            if chapter is not None:
                                chapter += 1
                            else:
                                chapter = 0
                            #print("chapter ", chapter)
                            text_dict[liber][chapter] = list()
                            for ellll in elll.getchildren():
                                typeof = check_type(ellll)
                                raw_text = ellll.text
                                #print("chapter ", chapter)                           
                                text_dict[liber][chapter].append([year, typeof, process_vrt(raw_text)])
            #chapters under liber 0-1
            elif ell.attrib["class"] == "chapter":
                if chapter is not None:
                    chapter += 1
                else:
                    chapter = 0
                text_dict[liber][chapter] = list()
                for elll in ell.getchildren():
                    typeof = check_type(elll)
                    raw_text = elll.text
                    text_dict[liber][chapter].append([year, typeof, process_vrt(raw_text)])

In [6]:
docs_by_year = [] 
docs_by_liber = []
docs_by_chapter = [] # list of chapters (liber, chapter, year, tokens)
all_by_chapter= [] #all chaps
txt_chapt_numbers = []
txt_chapt_num_current = []
text_to_all_chapters = {} #{chapter : {liber:chapter}} #only texts
text_dict_joined = {}

i = 0
for liber in text_dict.keys():
    if liber not in text_dict_joined.keys():
        text_dict_joined[liber] = {}
    for chapter in text_dict[liber].keys():
        if chapter not in text_dict_joined[liber].keys():
            text_dict_joined[liber][chapter] = {"year":None}
            
        
        chapter_tokens_txt = []
        chapter_tokens = []
        
        for txt_arr in text_dict[liber][chapter]:
            text_dict_joined[liber][chapter]["year"] = txt_arr[0]
            if txt_arr[1] not in text_dict_joined[liber][chapter].keys():
                text_dict_joined[liber][chapter][txt_arr[1]] = []
            text_dict_joined[liber][chapter][txt_arr[1]].append(' '.join([tok[0] for tok in txt_arr[2]]))
            for tok in txt_arr[2]: #iterates over tokens
                chapter_tokens.append((liber, chapter, txt_arr[0], txt_arr[1], tok[0], tok[1], tok[2])) #liber,chap, year,token
            
            if txt_arr[1] == "text": #exclude head etc.
                if str(liber) + "_" + str(chapter) not in txt_chapt_num_current:
                    txt_chapt_num_current.append(str(liber) + "_" + str(chapter))
                    text_to_all_chapters[i] = {liber:chapter}
                    i+= 1
                for tok in txt_arr[2]: #iterates over tokens
                    chapter_tokens_txt.append((liber, chapter, txt_arr[0], tok[0], tok[1], tok[2])) #liber,chap, year,token
        if len(chapter_tokens_txt) > 0:
            docs_by_chapter.append(chapter_tokens_txt)
        all_by_chapter.append(chapter_tokens)

In [7]:
corpus_chapters = []
corpus_chapters_lemmas = [] #no uknown
corpus_chapters_lemmas_all = []
corpus_chapters_lemmas_year = []
year = 0
for chap in docs_by_chapter:
    doc = []
    doc_lemmas = []
    doc_lemmas_all = []
    
    for tok in chap:
        if tok[4] not in ["PUN", "SENT"]:
            doc.append(tok[3].lower())            
            if tok[5] is not None and tok[5] != 'UNKNOWN':
                doc_lemmas.append(tok[5].lower())
                year = tok[2]
            elif tok[5] is not None:
                if tok[5] == 'UNKNOWN':
                    doc_lemmas_all.append(tok[3].lower())
                else:
                    doc_lemmas_all.append(tok[5].lower())
    corpus_chapters_lemmas_year.append(year)
    corpus_chapters.append(doc)
    corpus_chapters_lemmas.append(doc_lemmas)
    corpus_chapters_lemmas_all.append(doc_lemmas_all)

In [8]:
corpus_chapters_lemmas_reduced = []
stoplist_lemma = ["-", "major", "nemo", "aliter", "nisi", "verum", "quis", "per", "vir", "teneo", "saltim", "velut", "ita", 
                  "quis", "do", "etenim", "vix", "bene", "vel","facio|fio", "quoniam", "se", "a", "de", "cum","tuus", 
                  "omnis", "ex", "sive", "post", "ego", "tu", "non", "quia", "eo", "sum", "qui", "ut", "ac", "is", 
                  "ille","ad","in","et","sed", "que", "deus", "tam", "hic", "homo", "tempus", "aut", "atque", "habeo",
                  "dum", "unus", "ipse", "nec", "quoque", "adeo", "aput", "alius", "nullus", "enim", "genus", "autem", 
                  "quidem", "res", "mille", "si", "plus", "possum", "tamen", "ubi", "sed", "multus", "idem", "tunc",
                  "propter", "fero", "tam", "ne", "noster", "nos", "dominus", "ago", "contra", "res", "inter", "etiam", 
                  "sub", "duo", "tres", "septem", "quattuor", "quinque", "sex", "septem", "octo", "novem", "decem", 
                  "viginti", "ceterus", "annus", "circa", "homo", "ab", "sus", "insuper", "ab", "suus", "pro", "alter", "seu", "item", "quidam",
                 "magis", "facio", "dico", "appello", "itaque", "jam", "credo", "dies", "magis", "siquidem"]
for chapter in corpus_chapters_lemmas:
    corpus_chapters_lemmas_reduced.append([tok for tok in chapter if tok not in stoplist_lemma])

In [10]:
# serialization
import pickle
with open("corpora/corpus_chapters", "wb") as f:
    pickle.dump(corpus_chapters, f)

with open("corpora/corpus_chapters_lemmas", "wb") as f:
    pickle.dump(corpus_chapters_lemmas, f)

with open("corpora/corpus_chapters_lemmas_reduced", "wb") as f:
    pickle.dump(corpus_chapters_lemmas_reduced, f)
    
with open("corpora/text_to_all_chapters", "wb") as f:
    pickle.dump(text_to_all_chapters, f)

with open("corpora/text_dict_joined", "wb") as f:
    pickle.dump(text_dict_joined, f)
    
with open("corpora/docs_by_chapter", "wb") as f:
    pickle.dump(docs_by_chapter, f)
with open("corpora/docs_by_year", "wb") as f:
    pickle.dump(docs_by_year, f)
with open("corpora/docs_by_liber", "wb") as f:
    pickle.dump(docs_by_liber, f)